In [ ]:
!pip install wandb

In [ ]:
import wandb
wandb.login()

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install transformers==4.17

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer, DataCollatorWithPadding, TrainingArguments, AutoModelForSequenceClassification, Trainer
import evaluate
import numpy as np
from sklearn.metrics import confusion_matrix

In [ ]:
our_dataset=load_dataset("arbml/BRAD")
our_dataset

In [ ]:
our_dataset = our_dataset.shuffle(seed = 42)
our_dataset = our_dataset["train"].select(range(11000))
our_dataset

In [ ]:
our_dataset=our_dataset.train_test_split(test_size=.2)
our_dataset

In [ ]:
our_dataset = our_dataset .filter(lambda x : len(x["review"]) < 513)
our_dataset

In [ ]:
# our_dataset = our_dataset.rename_column("labels", "label")

In [ ]:
set(our_dataset['train']["label"])

In [ ]:
#our_dataset = our_dataset.filter(lambda x : x["label"] in [0,1,3,4])
#our_dataset

In [ ]:
def change_labels(example):
  return { "labels" : 0 if example["label"] in [0,1]  else 1 if example["label"] in [2] else 2}

our_dataset = our_dataset.map(change_labels )
our_dataset

In [ ]:
set(our_dataset['train']["labels"])

In [ ]:
#our_dataset = our_dataset.shuffle(seed = 42)
our_dataset = our_dataset.remove_columns(["review_id","book_id","user_id","label"])
our_dataset

In [ ]:
checkpoint = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)




def tokenize_function(example):
    return tokenizer(example["review"], truncation=True)

tokenized_dataset = our_dataset.map(tokenize_function, batched = True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
tokenized_dataset

In [ ]:
checkpoint_path = "/content/drive/MyDrive/checkpoint-205500"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint_path, num_labels = 3)

In [ ]:
def compute_metrics(eval_preds):
    accuracy_metric = load_metric("accuracy")
    f1_metric = load_metric("f1")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="weighted")
    # confusion = confusion_matrix(labels, predictions)
    # cfm_metric = evaluate.load("BucketHeadP65/confusion_matrix")
    # confusion = cfm_metric.compute(references=labels, predictions=predictions)

    return {
        "accuracy": accuracy,
        "f1_score": f1,
        # "Confusion matrix": confusion,
    }

In [ ]:
training_args = TrainingArguments("/content/drive/MyDrive/checkpoints/taiseer",
                                  evaluation_strategy="epoch",
                                  per_device_train_batch_size= 16,
                                  per_device_eval_batch_size=16,
                                  overwrite_output_dir=True,
                                  save_steps=496,
                                  save_total_limit=1,
                                  report_to="wandb"


                                  )

In [ ]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,)

In [ ]:
trainer.train()